# Análisis Comparativo: Fintechs LATAM

Comparativa de performance y valuación de las principales fintechs latinoamericanas cotizantes.

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta

pd.options.display.float_format = '{:.2f}'.format

## 1. Obtención de datos

In [ ]:
# Tickers de fintechs LATAM
TICKERS = {
    'MELI': 'MercadoLibre',
    'NU': 'Nubank',
    'DLO': 'DLocal',
    'STNE': 'StoneCo'
}

# Benchmark
BENCHMARK = '^GSPC'  # S&P 500

# Período de análisis
START_DATE = '2021-01-01'
END_DATE = datetime.now().strftime('%Y-%m-%d')

print(f"Período: {START_DATE} a {END_DATE}")

In [ ]:
def fetch_price_data(tickers, start, end):
    """Descarga precios ajustados de Yahoo Finance."""
    all_tickers = list(tickers.keys()) + [BENCHMARK]
    data = yf.download(all_tickers, start=start, end=end, progress=False)
    prices = data['Adj Close']
    return prices

prices = fetch_price_data(TICKERS, START_DATE, END_DATE)
prices = prices.dropna()

print(f"Datos obtenidos: {len(prices)} días de trading")
prices.tail()

## 2. Performance histórica

In [ ]:
# Retornos normalizados (base 100)
normalized = (prices / prices.iloc[0]) * 100

fig = px.line(
    normalized,
    title='Performance Normalizada (Base 100)',
    labels={'value': 'Valor', 'variable': 'Ticker', 'Date': 'Fecha'}
)

fig.update_layout(
    hovermode='x unified',
    legend=dict(orientation='h', yanchor='bottom', y=1.02),
    template='plotly_white'
)

fig.show()

In [ ]:
# Retornos diarios
returns = prices.pct_change().dropna()

# Métricas de performance
def calculate_metrics(returns_df, risk_free=0.05):
    """Calcula métricas de performance anualizadas."""
    trading_days = 252
    rf_daily = risk_free / trading_days
    
    metrics = pd.DataFrame(index=returns_df.columns)
    
    # Retorno anualizado
    metrics['Retorno Anual'] = returns_df.mean() * trading_days
    
    # Volatilidad anualizada
    metrics['Volatilidad'] = returns_df.std() * np.sqrt(trading_days)
    
    # Sharpe Ratio
    excess_returns = returns_df.mean() - rf_daily
    metrics['Sharpe'] = (excess_returns * trading_days) / (returns_df.std() * np.sqrt(trading_days))
    
    # Max Drawdown
    cumulative = (1 + returns_df).cumprod()
    rolling_max = cumulative.expanding().max()
    drawdowns = cumulative / rolling_max - 1
    metrics['Max Drawdown'] = drawdowns.min()
    
    # Sortino (downside deviation)
    negative_returns = returns_df.copy()
    negative_returns[negative_returns > 0] = 0
    downside_std = negative_returns.std() * np.sqrt(trading_days)
    metrics['Sortino'] = (metrics['Retorno Anual'] - risk_free) / downside_std
    
    return metrics.round(3)

metrics = calculate_metrics(returns)
metrics.sort_values('Sharpe', ascending=False)

In [ ]:
# Gráfico de riesgo-retorno
fig = px.scatter(
    metrics.reset_index(),
    x='Volatilidad',
    y='Retorno Anual',
    text='index',
    title='Riesgo vs Retorno',
    labels={'Volatilidad': 'Volatilidad Anual', 'Retorno Anual': 'Retorno Anual'}
)

fig.update_traces(
    textposition='top center',
    marker=dict(size=15)
)

fig.add_hline(y=0, line_dash='dash', line_color='gray')
fig.update_layout(template='plotly_white')
fig.show()

## 3. Matriz de correlación

In [ ]:
# Correlación de retornos
corr_matrix = returns[list(TICKERS.keys())].corr()

fig = px.imshow(
    corr_matrix,
    text_auto='.2f',
    color_continuous_scale='RdBu_r',
    zmin=-1, zmax=1,
    title='Matriz de Correlación - Fintechs LATAM'
)

fig.update_layout(template='plotly_white')
fig.show()

## 4. Datos fundamentales

In [ ]:
def get_fundamentals(tickers):
    """Obtiene métricas fundamentales de yfinance."""
    data = []
    
    for ticker, name in tickers.items():
        try:
            stock = yf.Ticker(ticker)
            info = stock.info
            
            data.append({
                'Ticker': ticker,
                'Empresa': name,
                'Market Cap (B)': info.get('marketCap', 0) / 1e9,
                'Revenue (B)': info.get('totalRevenue', 0) / 1e9,
                'P/S': info.get('priceToSalesTrailing12Months'),
                'P/E': info.get('trailingPE'),
                'EV/Revenue': info.get('enterpriseToRevenue'),
                'Profit Margin': info.get('profitMargins'),
                'Revenue Growth': info.get('revenueGrowth'),
            })
        except Exception as e:
            print(f"Error con {ticker}: {e}")
    
    return pd.DataFrame(data).set_index('Ticker')

fundamentals = get_fundamentals(TICKERS)
fundamentals

In [ ]:
# Comparativa de valuación
valuation_metrics = ['P/S', 'EV/Revenue', 'P/E']
valuation_df = fundamentals[valuation_metrics].dropna(how='all')

fig = px.bar(
    valuation_df.reset_index().melt(id_vars='Ticker'),
    x='Ticker',
    y='value',
    color='variable',
    barmode='group',
    title='Métricas de Valuación',
    labels={'value': 'Múltiplo', 'variable': 'Métrica'}
)

fig.update_layout(template='plotly_white')
fig.show()

## 5. Análisis de drawdowns

In [ ]:
def calculate_drawdowns(prices_df):
    """Calcula drawdowns históricos."""
    cumulative = prices_df / prices_df.iloc[0]
    rolling_max = cumulative.expanding().max()
    drawdowns = (cumulative / rolling_max - 1) * 100
    return drawdowns

drawdowns = calculate_drawdowns(prices[list(TICKERS.keys())])

fig = px.area(
    drawdowns,
    title='Drawdowns Históricos (%)',
    labels={'value': 'Drawdown (%)', 'variable': 'Ticker'}
)

fig.update_layout(
    hovermode='x unified',
    template='plotly_white'
)
fig.show()

## 6. Resumen ejecutivo

In [ ]:
print("="*60)
print("RESUMEN: FINTECHS LATAM")
print("="*60)
print(f"\nPeríodo analizado: {START_DATE} a {END_DATE}")
print(f"\nMEJOR SHARPE RATIO: {metrics['Sharpe'].idxmax()} ({metrics['Sharpe'].max():.2f})")
print(f"MENOR DRAWDOWN: {metrics['Max Drawdown'].idxmax()} ({metrics['Max Drawdown'].max():.1%})")
print(f"MAYOR RETORNO: {metrics['Retorno Anual'].idxmax()} ({metrics['Retorno Anual'].max():.1%})")
print(f"\nCORRELACIÓN PROMEDIO ENTRE FINTECHS: {corr_matrix.values[np.triu_indices(len(corr_matrix), k=1)].mean():.2f}")
print("="*60)